In [1]:
import tweepy
from twarc import Twarc2, expansions
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
from nltk.stem.porter import PorterStemmer as PS
import time
import schedule
import datetime

In [2]:
config = pd.read_csv("./config_academic.csv")

In [3]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]
twitterBearerToken=config['twitterApiBearerToken'][0]

In [5]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
client = tweepy.Client(auth, wait_on_rate_limit = True)

AttributeError: module 'tweepy' has no attribute 'Client'

In [22]:
# キーワードからツイートを取得
search_word='btc'
env='sentienv'
since='202105010000'
until='202105020000'
numTweets=500000

tweets = tweepy.Cursor(api.search_full_archive,  
                        query=search_word,
                        environment_name=env,
                        fromDate=since,
                        toDate=until
                        ).items(numTweets);


In [23]:
#ツイートの配列
t_data = []
for tweet in tweets:
    t_data.append([tweet.user.name,'@'+tweet.user.screen_name,tweet.user.friends_count,tweet.user.followers_count,tweet.text.replace('\n',''),tweet.favorite_count,tweet.retweet_count,tweet.created_at])
    
print("{}件収集\n".format(len(t_data)))    

TweepError: {'message': 'Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.', 'sent': '2021-08-29T14:29:19+00:00', 'transactionId': '97c4efcac3b47adc'}

In [ ]:
df = pd.DataFrame(data=t_data, columns=['Name','User Name','Following','Followers','Tweet','Favorite','RT','created at'])
df.head(10)

In [ ]:
df.tail(10)

In [12]:
#中身の確認
for tweet in tweepy.Cursor(api.search_full_archive, query='緊急事態宣言',environment_name=env,).items(10):
    print(tweet)


Status(_api=<tweepy.api.API object at 0x0000027CB0E1D580>, _json={'created_at': 'Sun Aug 22 05:15:39 +0000 2021', 'id': 1429311281996070919, 'id_str': '1429311281996070919', 'text': 'RT @NOSUKE0607: 政府「緊急事態宣言」拡大をまたも後手後手のダラダラ。来週初めに判断？その理由がすごいんですよ↓\n\n政府は９月１２日での解除を堅持したい考え。期限がずれ込むと、その後に予定される自民党総裁選や衆院選などの政治日程に影響を及ぼしかねないためだ。 ht…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1082452411828953088, 'id_str': '1082452411828953088', 'name': 'とく', 'screen_name': '6RVd2qFij4ZzbW2', 'location': '日本 東京', 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 215, 'friends_count': 131, 'listed_count': 0, 'favourites_count': 48805, 'statuses_count': 16862, 'created_at': 'Tue Jan 08 01:42:20 +00

In [9]:
df.to_csv("test.csv")

In [5]:
#lemmatization
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence

2021-07-23 14:02:36 INFO: Downloading default packages for language: en (English)...
2021-07-23 14:02:37 INFO: File exists: C:\Users\zxxxs\stanza_resources\en\default.zip.
2021-07-23 14:02:40 INFO: Finished downloading models and saved to C:\Users\zxxxs\stanza_resources.
2021-07-23 14:02:40 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-07-23 14:02:40 INFO: Use device: cpu
2021-07-23 14:02:40 INFO: Loading: tokenize
2021-07-23 14:02:40 INFO: Loading: pos
2021-07-23 14:02:40 INFO: Loading: lemma
2021-07-23 14:02:40 INFO: Loading: depparse
2021-07-23 14:02:40 INFO: Loading: sentiment
2021-07-23 14:02:40 INFO: Loading: ner
2021-07-23 14:02:41 INFO: Done loading processors!
C:\Users\zxxxs\anaconda3\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors a

In [6]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')
doc = nlp('Her bags are better than his.')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

2021-07-23 14:02:41 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-07-23 14:02:41 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-07-23 14:02:41 INFO: Use device: cpu
2021-07-23 14:02:41 INFO: Loading: tokenize
2021-07-23 14:02:41 INFO: Loading: pos
2021-07-23 14:02:41 INFO: Loading: lemma
2021-07-23 14:02:41 INFO: Done loading processors!


word: Her 	lemma: her
word: bags 	lemma: bag
word: are 	lemma: be
word: better 	lemma: good
word: than 	lemma: than
word: his 	lemma: he
word: . 	lemma: .


In [26]:
#lemmatization
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def lemma(txt):
    s = ''
    doc = nlp(txt)
    
    for sent in doc.sentences:
        for word in sent.words:
            s += word.lemma + ' '
    
    return s

2021-07-23 15:09:25 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-07-23 15:09:25 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-07-23 15:09:25 INFO: Use device: cpu
2021-07-23 15:09:25 INFO: Loading: tokenize
2021-07-23 15:09:25 INFO: Loading: pos
2021-07-23 15:09:25 INFO: Loading: lemma
2021-07-23 15:09:25 INFO: Done loading processors!


In [29]:
df['exTweet'] = df['Tweet'].apply(lemma)
df.to_csv("test.csv")
df.head(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,exTweet
0,Yin --- 👽🐐😈🐘🤏🎈,@AlienBoyYin,1307,1450,@khateva Forgot the third-FUCK NO ETH! lol,0,0,@khateva forget the third - fuck no eth ! lol
1,$uddenly $atoshi,@SuddenlySatoshi,533,2117,Motley Fool recommended Ethereum for their Bla...,0,0,motley fool recommend Ethereum for they blast ...
2,thinkreallyhard,@thinkreallyhar1,461,223,@andrwwang @dsesto94 @FutureNow_Art Dmx.eth 😂,1,0,@andrwwang @dsesto94 @ FutureNow_ art Dmx.eth 😂
3,J U N N A I,@Crack30041,24,11,X3 EARNING WITH LAST CALL💸💸What a beast 💥✅SUBS...,0,0,X3 ear with last call 💸 '' what a beast be sub...
4,mrkennybones,@mrkennybones,14,4,@BigBang_Crypto @mmbpng @intocryptoverse What ...,0,0,@BigBang_ crypto @mmbpng @intocryptoverse what...
5,Rixx: Trading Assistant,@RixxTech,29,56,1 hour top movers report #blockchain #crypto #...,1,0,1 hour top mover report # blockchain # crypto ...
6,Rixx: Trading Assistant,@RixxTech,29,56,4 hour top movers report #blockchain #crypto #...,1,0,4 hour top mover report # blockchain # crypto ...
7,TradeKing👑,@tradeking86,1078,1535,Example #eth monthly #chart…so its presenting ...,7,1,example # eth monthly # chart … so its present...
8,🐍 EtherMamba 🐍,@eth_gang,512,108,@DrippyJesusNFT you’re not on opensea?,2,0,@drippyjesusnft you be not on opensea ?
9,🐍 EtherMamba 🐍,@eth_gang,512,108,@DrippyJesusNFT Following .. will support and ...,2,0,@drippyjesusnft follow .. will support and buy...


In [29]:
import time
import schedule
import datetime

def job():
    #検索ワード、条件
    s='btc min_faves:5 -filter:retweets'
    
    #5日前と4日前の時刻を取得
    dt  = datetime.date.today()
    dt5 = dt + datetime.timedelta(days=-5)
    dt4 = dt + datetime.timedelta(days=-4)

    tweets = tweepy.Cursor(api.search,  
                           q=s,
                           exclude_replies=True,
                           tweet_mode='extended',
                           lang = 'en',
                           since= dt5,
                           until= dt4,
                          ).items(10);
    
    #ツイートのリスト
    t_data = []
    for tweet in tweets:
        t_data.append([tweet.user.name,
                       '@'+tweet.user.screen_name,
                       tweet.user.friends_count,
                       tweet.user.followers_count,
                       tweet.full_text.replace('\n',''),
                       tweet.favorite_count,
                       tweet.retweet_count,
                       tweet.created_at])


              
    #処理前ツイート出力
    df = pd.DataFrame(data=t_data, columns=['Name','User Name','Following','Followers','Tweet','Favorite','RT','created at'])
    
    df.to_csv(f'./tweet-of-btc/{dt5}-{dt4}.csv')

    #件数確認
    print('{} ~ {}：{}件\n'.format(dt5,dt4,len(t_data)))

In [7]:
import time
import schedule
import datetime

# 現在時刻
dt  = datetime.date.today()
dt5 = dt + datetime.timedelta(days=-5)
dt4 = dt + datetime.timedelta(days=-4)


print(dt,dt5,dt4)



2021-08-03 2021-07-29 2021-07-30


In [30]:
def main():
    schedule.every(1).minutes.do(job)

    while True:
        schedule.run_pending()
        time.sleep(1)
        
if __name__ == "__main__":
    main()

1627995482.176932
1627995487.2204337
1627995492.2688885
1627995497.320142


KeyboardInterrupt: 

In [8]:
df = pd.DataFrame(columns=['現在時刻', '区間(start)','区間(end)','収集件数'])
df.to_csv(f'./tweet-of-btc/assembling-log.csv')

In [7]:
dt  = datetime.date.today()
dt5 = dt + datetime.timedelta(days=-5)
dt4 = dt + datetime.timedelta(days=-4)

r = {dt5} + ' ~ ' + {dt4}  

s = pd.Series([dt,r,dt4], index=['現在時刻', '区間(start)','区間(end)','収集件数'])
df = df.append(s, ignore_index=True)
df.to_csv(f'./tweet-of-btc/assembling-log.csv')

TypeError: unsupported operand type(s) for +: 'set' and 'str'

In [12]:
import csv

rawdata = [["twitterApiKey", "twitterApiSecret", "twitterApiAccessToken","twitterApiAccessTokenSecret","twitterApiBearerToken"],
           ["ZxWzg0n3DGlchNv4X3zlARA1s","EkwRlNfz7Z5xIb8UDTwQXYiQl18QZG95WKGyMINcSsRRzF4pwO","1420597303946059782-xjsjzJZYuIps4E7OhrBnyjVmNopNmw","Yqi0mohf7WdM74H2SZdbm11cgm3bgAOO3zJmfo4JwDaRx","AAAAAAAAAAAAAAAAAAAAAOvfSQEAAAAAgbIvli1%2BVfq4kSlsPf4WbH9WDfY%3DLktjvti1ZBhDAq4oMkX9ibYErI15gQlHiHVqmy9hPDjdQvnnv2"]]

# CSVファイルを作成
with open("config_academic.csv", mode="w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(rawdata)

In [18]:
from datetime import timezone
t=datetime.timedelta(timezone('Asia/Tokyo'))
print(t)

TypeError: timezone() argument 1 must be datetime.timedelta, not str